In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[7],5
commitnumber,1013f91,a06b618
conc,-,-
dv,-,0.001
klin,-,1e-24
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[3]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`              
            layer        coolrg                layer        coolrg
pressure                                                          
0.000312        1 -3.581966e-06                    1  3.584831e-06
0.000750        2 -3.591365e-06                    2  1.307246e-05
0.001052        3 -3.615405e-06                    3 -4.266574e-07
0.001476        4 -3.637743e-06                    4 -2.692173e-06
0.002070        5 -3.655963e-06                    5 -2.118810e-06
0.002904        6 -3.675053e-06                    6 -2.715195e-06
0.004074        7 -3.691354e-06                    7 -2.364270e-06
0.005714        8 -3.705688e-06                    8 -2.718045e-06
0.008015        9 -3.717069e-06                    9 -2.514213e-06
0.011243       10 -3.726846e-06                   10 -2.670929e-06
0.015771       11 -3.733835e-06                   11 -2.709710e-06
0.022122       12 -3.738441e-06                   12 -2.695528e-06
0.031031       13 -3.740082e-06                   13 -2.693247e-06
0.043528       14 -3.738834e-06                   14 -1.308122e-06
0.061057       15 -3.714160e-06                   15  1.213534e-07
0.085645       16 -3.459690e-06                   16 -8.482670e-07
0.120136       17 -2.432300e-06                   17 -1.858146e-06
0.168516       18 -1.151920e-07                   18 -1.322263e-06
0.236378       19  4.229040e-06                   19  1.311890e-07
0.331549       20  1.167828e-05                   20  2.825558e-06
0.465100       21  2.374184e-05                   21  7.352207e-06
0.652400       22  4.236787e-05                   22  1.461009e-05
0.915100       23  6.339370e-05                   23  2.272213e-05
1.283650       24  6.529073e-05                   24  2.367817e-05
1.800600       25  4.803166e-05                   25  1.714108e-05
2.525700       26  3.098397e-05                   26  1.036869e-05
3.542800       27  1.835294e-05                   27  5.304279e-06
4.969550       28  9.720536e-06                   28  1.877455e-06
6.970850       29  4.159946e-06                   29 -2.972831e-07
9.778100       30  6.754996e-07                   30 -1.652420e-06
13.715850      31 -1.551211e-06                   31 -2.506657e-06
19.239350      32 -2.900351e-06                   32 -3.004957e-06
26.987250      33 -3.562794e-06                   33 -3.255605e-06
37.855300      34 -3.872528e-06                   34 -3.404094e-06
53.100050      35 -4.053650e-06                   35 -3.530699e-06
73.887500      36 -4.164267e-06                   36 -3.657519e-06
97.662500      37 -4.246311e-06                   37 -3.793973e-06
121.437500     38 -4.338230e-06                   38 -3.908209e-06
145.212500     39 -4.428323e-06                   39 -3.944846e-06
168.987500     40 -4.721787e-06                   40 -4.219637e-06
192.762500     41 -7.444643e-06                   41 -6.696998e-06
216.537500     42 -1.974888e-05                   42 -1.882358e-05
240.312500     43 -4.196204e-05                   43 -4.292587e-05
264.087500     44 -7.295605e-05                   44 -8.163511e-05
287.862500     45 -1.036296e-04                   45 -1.318431e-04
311.637500     46 -9.591706e-05                   46 -1.535238e-04
335.412500     47 -4.595623e-05                   47 -1.414342e-04
359.187500     48  5.663469e-05                   48 -9.166711e-05
382.962500     49  2.309759e-04                   49  1.539007e-05
406.737500     50  4.972050e-04                   50  2.018926e-04
430.512500     51  8.751792e-04                   51  4.870663e-04
454.287500     52  1.357261e-03                   52  8.783727e-04
478.062500     53  1.958066e-03                   53  1.393200e-03
501.837500     54  2.706122e-03                   54  2.056729e-03
525.612500     55  3.623675e-03                   55  2.861916e-03
549.387500     56  4.735060e-03                   56  3.853446e-03
573.162500     57  6.070877e-03                   57  5.05874

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -30.904959  0.000000e+00 -30.904959 -30.945951   
0.000624    2     -30.904959  9.990577e-11 -30.904959 -30.945951   
0.000876    3     -30.904959  1.408654e-10 -30.904959 -30.945951   
0.001229    4     -30.904959  2.005144e-10 -30.904959 -30.945951   
0.001723    5     -30.904959  2.875950e-10 -30.904959 -30.945951   
0.002417    6     -30.904959  4.146906e-10 -30.904959 -30.945951   
0.003391    7     -30.904959  6.002059e-10 -30.904959 -30.945951   
0.004757    8     -30.904959  8.709495e-10 -30.904959 -30.945951   
0.006672    9     -30.904959  1.266001e-09 -30.904959 -30.945951   
0.009359    10    -30.904959  1.842385e-09 -30.904959 -30.945951   
0.013128    11    -30.904959  2.683227e-09 -30.904959 -30.945951   
0.018415    12    -30.904959  3.909736e-09 -30.904959 -30.945951   
0.025830    13    -30.904959  5.698602e-09 -30.904959 -30.945951   
0.036232    14    -30.904959  8.307451e-09 -30.904959 -30.945951   
0.050823    15    -30.904959  1.212136e-08 -30.904959 -30.945951   
0.071291    16    -30.904959  1.815829e-08 -30.904959 -30.945951   
0.100000    17    -30.904959  2.979447e-08 -30.904959 -30.945951   
0.140271    18    -30.904959  5.506694e-08 -30.904959 -30.945951   
0.196760    19    -30.904959  1.097736e-07 -30.904959 -30.945951   
0.275997    20    -30.904959  2.267917e-07 -30.904959 -30.945951   
0.387100    21    -30.904959  4.740697e-07 -30.904959 -30.945951   
0.543100    22    -30.904959  9.911822e-07 -30.904958 -30.945951   
0.761700    23    -30.904959  2.055454e-06 -30.904957 -30.945951   
1.068500    24    -30.904959  4.058827e-06 -30.904955 -30.945951   
1.498800    25    -30.904958  6.931946e-06 -30.904952 -30.945951   
2.102400    26    -30.904958  1.014466e-05 -30.904948 -30.945952   
2.949000    27    -30.904959  1.347202e-05 -30.904945 -30.945952   
4.136600    28    -30.904960  1.684535e-05 -30.904943 -30.945953   
5.802500    29    -30.904961  2.024449e-05 -30.904941 -30.945954   
8.139200    30    -30.904964  2.369409e-05 -30.904939 -30.945956   
11.417000   31    -30.904967  2.724034e-05 -30.904939 -30.945959   
16.014700   32    -30.904971  3.087942e-05 -30.904941 -30.945962   
22.464000   33    -30.904978  3.460736e-05 -30.904943 -30.945966   
31.510500   34    -30.904985  3.862348e-05 -30.904946 -30.945972   
44.200100   35    -30.904996  4.324908e-05 -30.904952 -30.945980   
62.000000   36    -30.905009  4.867471e-05 -30.904961 -30.945991   
85.775000   37    -30.905028  5.484119e-05 -30.904972 -30.946005   
109.550000  38    -30.905045  6.057662e-05 -30.904984 -30.946020   
133.325000  39    -30.905063  6.630443e-05 -30.904996 -30.946035   
157.100000  40    -30.905081  7.205838e-05 -30.905009 -30.946050   
180.875000  41    -30.905100  7.835235e-05 -30.905022 -30.946066   
204.650000  42    -30.905133  9.010991e-05 -30.905043 -30.946093   
228.425000  43    -30.905232  1.324757e-04 -30.905099 -30.946176   
252.200000  44    -30.905478  2.607237e-04 -30.905217 -30.946387   
275.975000  45    -30.906014  5.909610e-04 -30.905422 -30.946850   
299.750000  46    -30.907049  1.334211e-03 -30.905714 -30.947754   
323.525000  47    -30.908590  2.606142e-03 -30.905985 -30.949110   
347.300000  48    -30.910680  4.565970e-03 -30.906114 -30.950945   
371.075000  49    -30.913429  7.475451e-03 -30.905954 -30.953366   
394.850000  50    -30.916960  1.165604e-02 -30.905304 -30.956499   
418.625000  51    -30.921391  1.748710e-02 -30.903904 -30.960447   
442.400000  52    -30.926843  2.540453e-02 -30.901439 -30.965302   
466.175000  53    -30.933302  3.568549e-02 -30.897616 -30.971072   
489.950000  54    -30.940790  4.868848e-02 -30.892102 -30.977800   
513.725000  55    -30.949385  6.490495e-02 -30.884480 -30.985580   
537.500000  56    -30.959157  8.488219e-02 -30.874275 -30.994408   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')